In [5]:
### Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf 

In [6]:
### Load in Stock Data
tickers = 'GOOG'
start_date = '2024-01-01'

def import_stock_data(tickers, start_date):
    # Check if tickers is a list or a single ticker string
    if isinstance(tickers, list):
        data = yf.download(tickers, start=start_date)[['Close', 'High', 'Low']]
    else:
        data = yf.download(tickers, start=start_date)[['Close', 'High', 'Low']]
        data = pd.DataFrame(data)

    # Reset index to make 'Date' a column instead of index
    data.reset_index(inplace=True)
    data['Date'] = pd.to_datetime(data['Date'])

    return data

stock_data = import_stock_data(tickers, start_date)
stock_data

[*********************100%%**********************]  1 of 1 completed


,Date,Close,High,Low
0,2024-01-02,139.559998,140.615005,137.740005
1,2024-01-03,140.360001,141.089996,138.429993
2,2024-01-04,138.039993,140.634995,138.009995
3,2024-01-05,137.389999,138.809998,136.850006
4,2024-01-08,140.529999,140.639999,137.880005
...,...,...,...,...
72,2024-04-16,156.000000,157.229996,155.050003
73,2024-04-17,156.880005,158.681000,156.134995
74,2024-04-18,157.460007,158.485001,156.210007
75,2024-04-19,155.720001,157.990005,153.910004


In [11]:
def get_ADX(data, window):
    # Calculate True Range (TR)
    tr1 = pd.DataFrame(data['High'] - data['Low'])
    tr2 = pd.DataFrame(abs(data['High'] - data['Close'].shift(1)))
    tr3 = pd.DataFrame(abs(data['Low'] - data['Close'].shift(1)))
    frames = [tr1, tr2, tr3]
    data['TR'] = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)

    # Calculate the Directional Movement (DM +/-) = Current High/Low - Previous High/Low (if both are positive, otherwise zero)
    for i in range(1, len(data)):
        # Build positive and negative DM columns
        pdm = data.iloc[i]['High'] - data.iloc[i - 1]['High']
        ndm = data.iloc[i - 1]['Low'] - data.iloc[i]['Low']
    
        data.at[i, 'PDM'] = pdm if pdm > 0 else 0
        data.at[i, 'NDM'] = abs(ndm) if ndm > 0 else 0

    # Smooth TR, PDM, and NDM using 14-period EMA
    data['TR_Smooth'] = data['TR'].ewm(span = window, adjust = False).mean()
    data['PDM_Smooth'] = data['PDM'].ewm(span = window, adjust = False).mean()
    data['NDM_Smooth'] = data['NDM'].ewm(span = window, adjust = False).mean()

    # Calculate the Directional Indicators (DI+ and DI-) = (PDM / Smoothed TR) * 100 and = (NDM / Smoothed TR) * 100
    data['DI+'] = (data['PDM_Smooth'] / data['TR_Smooth']) * 100
    data['DI-'] = (data['NDM_Smooth'] / data['TR_Smooth']) * 100

    # Calculate the Directional Index (DX) = (abs(DI+ - DI-) / (DI+ + DI-)) * 100
    data['DX'] = (abs(data['DI+'] - data['DI-']) / (data['DI+'] + data['DI-'])) * 100

    # Calculate Average Directional Index (ADX) = 14-period EMA of DX
    data['ADX'] = data['DX'].ewm(span = window, adjust = False).mean()

    return data

# Function Call
single_ticker = get_ADX(stock_data, window = 14)
single_ticker.tail()

,Date,Close,High,Low,TR,PDM,NDM,TR_Smooth,PDM_Smooth,NDM_Smooth,DI+,DI-,DX,ADX
72,2024-04-16,156.000000,157.229996,155.050003,2.179993,0.000000,1.099991,3.240541,0.950544,0.672822,29.332883,20.762653,17.107772,41.179560
73,2024-04-17,156.880005,158.681000,156.134995,2.681000,1.451004,0.000000,3.165936,1.017272,0.583113,32.131801,18.418336,27.128442,39.306077
74,2024-04-18,157.460007,158.485001,156.210007,2.274994,0.000000,0.000000,3.047143,0.881636,0.505364,28.933192,16.584855,27.128442,37.682392
75,2024-04-19,155.720001,157.990005,153.910004,4.080002,0.000000,2.300003,3.184858,0.764084,0.744649,23.991162,23.380933,1.288160,32.829828
76,2024-04-22,157.949997,159.184998,155.660004,3.524994,1.194992,0.000000,3.230209,0.821539,0.645363,25.432988,19.978980,12.010067,30.053860
